In [33]:
import kerchunk
import xarray as xr
from kerchunk.hdf import SingleHdf5ToZarr
from kerchunk.combine import MultiZarrToZarr
import fsspec
from glob import glob
import xradar as xd
import matplotlib.pyplot as plt
import cmweather
import numpy as np
import hvplot.xarray
import pyart
from xarray.core.datatree import DataTree


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [3]:
def fix_angle(ds):
    """
    Aligns the radar volumes
    """
    ds["time"] = ds.time.load()  # Convert time from dask to numpy

    start_ang = 0  # Set consistent start/end values
    stop_ang = 360

    # Find the median angle resolution
    angle_res = np.round(ds.azimuth.diff("azimuth").median(),2)
    
    # Determine whether the radar is spinning clockwise or counterclockwise
    median_diff = ds.azimuth.diff("time").median()
    ascending = median_diff > 0
    direction = 1 if ascending else -1

    # first find exact duplicates and remove
    ds = xd.util.remove_duplicate_rays(ds)

    # second reindex according to retrieved parameters
    ds = xd.util.reindex_angle(
        ds, start_ang, stop_ang, angle_res, direction, method="nearest"
    )

    ds = ds.expand_dims("volume_time")  # Expand for volumes for concatenation

    ds["volume_time"] = [np.nanmin(ds.time.values)]

    return ds

In [19]:
erad_files = glob("/media/alfonso/drive/Laura/ERAD/20240522_MeteoSwiss_ARPA_Lombardia/Data/Cband/MonteLema*.nc")
xband_files =glob("/media/alfonso/drive/Laura/ERAD/20240522_MeteoSwiss_ARPA_Lombardia/Data/Xband/DES_VOL_RAW*.nc")
xband_files[:1]

['/media/alfonso/drive/Laura/ERAD/20240522_MeteoSwiss_ARPA_Lombardia/Data/Xband/DES_VOL_RAW_20240522_1745.nc']

In [55]:
radar = pyart.io.read_cfradial(xband_files[0])
list(radar.fields)


['DBZ_TOT',
 'DBZ',
 'VEL',
 'WIDTH',
 'ZDR',
 'KDP',
 'RHOHV',
 'PHIDP',
 'HCLASS',
 'MCSSX']

In [58]:
radar.info("compact")

altitude: <ndarray of type: float64 and shape: (1,)>
altitude_agl: <ndarray of type: float64 and shape: (1,)>
antenna_transition: <ndarray of type: int8 and shape: (2880,)>
azimuth: <ndarray of type: float32 and shape: (2880,)>
elevation: <ndarray of type: float32 and shape: (2880,)>
fields:
	DBZ_TOT: <ndarray of type: float32 and shape: (2880, 416)>
	DBZ: <ndarray of type: float32 and shape: (2880, 416)>
	VEL: <ndarray of type: float32 and shape: (2880, 416)>
	WIDTH: <ndarray of type: float32 and shape: (2880, 416)>
	ZDR: <ndarray of type: float32 and shape: (2880, 416)>
	KDP: <ndarray of type: float32 and shape: (2880, 416)>
	RHOHV: <ndarray of type: float32 and shape: (2880, 416)>
	PHIDP: <ndarray of type: float32 and shape: (2880, 416)>
	HCLASS: <ndarray of type: float32 and shape: (2880, 416)>
	MCSSX: <ndarray of type: int16 and shape: (2880, 416)>
fixed_angle: <ndarray of type: float32 and shape: (8,)>
instrument_parameters:
	frequency: <ndarray of type: float32 and shape: (1,)>


In [62]:
xr.open_dataset(xband_files[0], engine='cfradial1')

<xarray.Dataset> Size: 525B
Dimensions:              (sweep: 8, frequency: 1)
Coordinates:
  * frequency            (frequency) float32 4B 9.339e+09
Dimensions without coordinates: sweep
Data variables: (12/13)
    sweep_fixed_angle    (sweep) float32 32B ...
    sweep_group_name     (sweep) <U9 288B ...
    volume_number        float64 8B ...
    platform_type        |S32 32B ...
    primary_axis         |S32 32B ...
    status_str           |S1 1B ...
    ...                   ...
    time_coverage_start  |S32 32B ...
    time_coverage_end    |S32 32B ...
    latitude             float64 8B ...
    longitude            float64 8B ...
    altitude             float64 8B ...
    altitude_agl         float64 8B ...
Attributes: (12/14)
    Conventions:         CF-1.7
    version:             CF-Radial-1.4
    title:               VOL_A
    institution:         
    references:          Conversion software: Radx::SigmetRadxFile
    source:              Sigmet IRIS software
    ...                  ...
    site_name:           Desio_Radar
    scan_name:           VOL_A
    scan_id:             0
    platform_is_mobile:  false
    ray_times_increase:  true
    history:             Wed May 22 18:13:42 2024: ncap2 -Os fixed_angle  (:)...

In [32]:
xr.open_dataset(xband_files[0], engine="cfradial1")

<xarray.Dataset> Size: 525B
Dimensions:              (sweep: 8, frequency: 1)
Coordinates:
  * frequency            (frequency) float32 4B 9.339e+09
Dimensions without coordinates: sweep
Data variables: (12/13)
    sweep_fixed_angle    (sweep) float32 32B ...
    sweep_group_name     (sweep) <U9 288B ...
    volume_number        float64 8B ...
    platform_type        |S32 32B ...
    primary_axis         |S32 32B ...
    status_str           |S1 1B ...
    ...                   ...
    time_coverage_start  |S32 32B ...
    time_coverage_end    |S32 32B ...
    latitude             float64 8B ...
    longitude            float64 8B ...
    altitude             float64 8B ...
    altitude_agl         float64 8B ...
Attributes: (12/14)
    Conventions:         CF-1.7
    version:             CF-Radial-1.4
    title:               VOL_A
    institution:         
    references:          Conversion software: Radx::SigmetRadxFile
    source:              Sigmet IRIS software
    ...                  ...
    site_name:           Desio_Radar
    scan_name:           VOL_A
    scan_id:             0
    platform_is_mobile:  false
    ray_times_increase:  true
    history:             Wed May 22 18:13:42 2024: ncap2 -Os fixed_angle  (:)...

In [5]:
groups = [
        i for i in list(dt.groups) if i.startswith("/sweep") if i not in ["/"]
    ]
groups[:3]

['/sweep_0', '/sweep_1', '/sweep_2']

In [6]:
radars = {"cband": "/media/alfonso/drive/Laura/ERAD/20240522_MeteoSwiss_ARPA_Lombardia/Data/Cband/MonteLema", 
          "xband": "/media/alfonso/drive/Laura/ERAD/20240522_MeteoSwiss_ARPA_Lombardia/Data/Xband/DES_VOL_RAW",
         }

In [16]:
root = {}
for key in list(radars.keys()):
    print(key)
    erad_files = sorted(glob(f"{radars[key]}*.nc"))
    dt = xd.io.open_cfradial1_datatree(erad_files[0])
    groups = [
        i[1:] for i in list(dt.groups) if i.startswith("/sweep") if i not in ["/"]
    ]
    groups[:3]
    for group in groups:
        print(group)
        ds = xr.open_mfdataset(
            erad_files[:3],
            preprocess=fix_angle,
            engine="cfradial1",
            group=group,
            concat_dim="volume_time",
            combine="nested",
        ).xradar.georeference()
        root[f"{key}/{group}"] = ds

cband
sweep_0
sweep_1
sweep_2
sweep_3
sweep_4
sweep_5
sweep_6
sweep_7
sweep_8
sweep_9
sweep_10
sweep_11
sweep_12
sweep_13
sweep_14
sweep_15
sweep_16
sweep_17
sweep_18
sweep_19
xband


ValueError: Dimensions {'n_points'} do not exist. Expected one or more of FrozenMappingWarningOnValuesAccess({'azimuth': 360, 'range': 416})

In [ ]:
DataTree.from_dict(root)

In [13]:
root

{'cband/sweep_0': <xarray.Dataset> Size: 51MB
 Dimensions:                              (volume_time: 3, azimuth: 360,
                                           range: 492)
 Coordinates:
   * range                                (range) float32 2kB 250.0 ... 2.457e+05
   * azimuth                              (azimuth) float32 1kB 0.5 1.5 ... 359.5
     time                                 (volume_time, azimuth) datetime64[ns] 9kB ...
     elevation                            (volume_time, azimuth) float32 4kB 6...
     latitude                             float32 4B 46.04
     longitude                            float32 4B 8.833
     altitude                             float32 4B 1.626e+03
   * volume_time                          (volume_time) datetime64[ns] 24B 202...
     crs_wkt                              int64 8B 0
     x                                    (azimuth, range, volume_time) float64 4MB ...
     y                                    (azimuth, range, volume_time) fl

In [ ]:
fig, ax = plt.subplots()
ds.isel(volume_time=1).reflectivity.plot(x="x", y="y", cmap="ChaseSpectral", vmin=-10, vmax=50)

In [ ]:
ds.to_zarr()

In [ ]:
ds.reflectivity.hvplot.quadmesh(x="x", 
                                y="y", 
                                groupby="volume_time", 
                                widget_type="scrubber",
                                widget_location="bottom",
                                clim=(-10, 50),
                                cmap="ChaseSpectral"
                                # rasterize=True, 
                               )